In [456]:
import os
from os import environ

import requests
import io

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt

In [457]:
Bcg=pd.read_csv('C:/Users/35266/Documents/Python Scripts/el/Spoil upside calculation database with new RPK.csv',sep=';')

Bcg=Bcg.loc[(Bcg['leg']=='CPH')&(Bcg['uplift_population']=='YES'),:]

Bcg.index=range(Bcg.shape[0])

Bcg=Bcg[['Departure Date','Flight Number','SEASON','Sum_ASK','Sum_RPK','New RPK']]

Bcg['LF_old']=Bcg['Sum_RPK']/Bcg['Sum_ASK']
Bcg['LF_new']=Bcg['New RPK']/Bcg['Sum_ASK']

Bcg=Bcg[[x for x in Bcg.columns if 'ASK' not in x and 'RPK' not in x]]

Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: pd.to_datetime(x.split('.')[2]+'-'+x.split('.')[1]+'-'+x.split('.')[0]))

Bcg['Month']=Bcg['Departure Date'].apply(lambda x: x.month)
Bcg['Weekday']=Bcg['Departure Date'].apply(lambda x: x.weekday())

Bcg=Bcg[[x for x in Bcg.columns if 'LF' not in x]+[x for x in Bcg.columns if 'LF' in x]]

In [458]:
Bcg_Mean=Bcg.groupby(['Flight Number','Month','Weekday'])['LF_old'].mean().reset_index()\
.merge(Bcg.groupby(['Flight Number','Month','Weekday'])['LF_new'].mean().reset_index(),on=['Flight Number','Month','Weekday'])\
.rename(columns={'LF_old': 'LF_old_mean', 'LF_new': 'LF_new_mean'})

Bcg_Var=Bcg.groupby(['Flight Number','Month','Weekday'])['LF_old'].var().reset_index()\
.merge(Bcg.groupby(['Flight Number','Month','Weekday'])['LF_new'].var().reset_index(),on=['Flight Number','Month','Weekday'])\
.rename(columns={'LF_old': 'LF_old_var', 'LF_new': 'LF_new_var'})

Bcg_N=Bcg.groupby(['Flight Number','Month','Weekday'])['LF_old'].size().reset_index()\
.merge(Bcg.groupby(['Flight Number','Month','Weekday'])['LF_new'].size().reset_index(),on=['Flight Number','Month','Weekday'])\
.rename(columns={'LF_old': 'LF_old_n', 'LF_new': 'LF_new_n'})

replace_value_old=np.mean(Bcg_Var['LF_old_var'])
replace_value_new=np.mean(Bcg_Var['LF_new_var'])

Bcg_Var['LF_old_var']=Bcg_Var['LF_old_var'].apply(lambda x: replace_value_old if pd.isnull(x) or x==0 else x)
Bcg_Var['LF_new_var']=Bcg_Var['LF_new_var'].apply(lambda x: replace_value_new if pd.isnull(x) or x==0 else x)

In [459]:
Raw=pd.concat([pd.read_csv('C:/Users/35266/Documents/Python Scripts/el/MAR18.csv',sep=';'),
pd.read_csv('C:/Users/35266/Documents/Python Scripts/el/JUN18.csv',sep=';'),
pd.read_csv('C:/Users/35266/Documents/Python Scripts/el/OCT18.csv',sep=';'),
pd.read_csv('C:/Users/35266/Documents/Python Scripts/el/JAN19.csv',sep=';')])

Raw['Calendar day']=Raw['Calendar day'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))
Raw['LF_actual']=Raw['Load Factor'].apply(lambda x: float(x.replace(',','.').replace('%',''))/100)
Raw['Direction']=Raw[['Sched Airport Depart','Sched Airport Arriva']].apply(lambda x: 'O' if x[0]=='LUX' and x[1]=='CPH' else 'I',axis=1)

Raw=Raw.rename(columns={'Calendar day': 'Departure Date'})
Raw=Raw[['Departure Date','Flight Number','Direction','LF_actual']]

Raw['Month']=Raw['Departure Date'].apply(lambda x: x.month)
Raw['Weekday']=Raw['Departure Date'].apply(lambda x: x.weekday())

Raw=Raw[[x for x in Raw.columns if 'LF' not in x]+[x for x in Raw.columns if 'LF' in x]]

In [460]:
Raw_Mean=Raw.groupby(['Flight Number','Month','Weekday'])['LF_actual'].mean().reset_index()\
.rename(columns={'LF_actual': 'LF_actual_mean'})
Raw_Var=Raw.groupby(['Flight Number','Month','Weekday'])['LF_actual'].var().reset_index()\
.rename(columns={'LF_actual': 'LF_actual_var'})
Raw_N=Raw.groupby(['Flight Number','Month','Weekday'])['LF_actual'].size().reset_index()\
.rename(columns={'LF_actual': 'LF_actual_n'})

replace_value_actual=np.mean(Raw_Var['LF_actual_var'])

Raw_Var['LF_actual_var']=Raw_Var['LF_actual_var'].apply(lambda x: replace_value_actual if pd.isnull(x) or x==0 else x)

In [461]:
Data=Bcg_Mean.merge(Bcg_Var,on=['Flight Number','Month','Weekday'])\
.merge(Bcg_N,on=['Flight Number','Month','Weekday'])\
.merge(Raw_Mean,on=['Flight Number','Month','Weekday'])\
.merge(Raw_Var,on=['Flight Number','Month','Weekday'])\
.merge(Raw_N,on=['Flight Number','Month','Weekday'])

In [464]:
# sorted(Data[['LF_actual_mean','LF_actual_var','LF_actual_n','LF_old_mean','LF_old_var','LF_old_n']]\
# .apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1))